In [67]:
import networkx as nx
import json 
import pandas
import matplotlib.pyplot as plt
import altair as alt
import nx_altair as nxa

alt.data_transformers.disable_max_rows() # No maximum dataset

DataTransformerRegistry.enable('default')

In [77]:
# Define the custom theme, modified from https://www.geeksforgeeks.org/setting-a-custom-color-theme-in-altair-as-default/
def brat():
    return {
        "config": {
            "view": {"continuousWidth": 400, "continuousHeight": 300},
            "mark": {"color": "steelblue"},
            "axis": {
                "labelFontSize": 12,
                "titleFontSize": 14,
                "labelColor": "gray",
                "titleColor": "black"
            },
            "range": {
                "category": ["#8ACE00","#D81E5B", "#35605A", "#420039", "#BDD4E7"]
            }
        }
    }

# Register and enable the custom theme
alt.themes.register('brat', brat)
alt.themes.enable('brat')

ThemeRegistry.enable('brat')

In [2]:
def open_file(path:str)->dict:
    cache_file = open(path, 'r')
    cache_contents = cache_file.read()
    artist_data = json.loads(cache_contents)
    cache_file.close()
    return artist_data

In [9]:
weighted_network = open_file('10_26_1.json')

In [61]:
# adding name for convenience
for artist in weighted_network.keys():
    weighted_network[artist]['name']=artist

In [63]:
g = nx.Graph()
for artist in weighted_network.keys():
    g.add_node(artist)
    colab = weighted_network[artist]['collaborators'].keys()
    for y in colab:
        if artist !=y:
            if y not in g.nodes:
                g.add_node(y)
            g.add_edge(artist,y)

for n in g.nodes():
    if n in weighted_network.keys():
        for key in weighted_network[n].keys():
            g.nodes[n][key]= weighted_network[n][key]
    else:
        g.nodes[n]['in_playlist']=False
        g.nodes[n]['name']=n

In [57]:
#weighted_network['Hanumankind'].keys()#['name'] #.keys()
g.nodes['Hanumankind'].keys()

dict_keys(['genres', 'img_info', 'popularity', 'followers', 'collaborators', 'in_playlist', 'link', 'name'])

In [54]:
from itertools import count
groups = set(nx.get_node_attributes(g,'in_playlist').values())
mapping = dict(zip(sorted(groups),count()))
colors = [mapping[g.nodes[n]['in_playlist']] for n in g.nodes()]

In [79]:
#nx.draw_kamada_kawai(g)
position = nx.spring_layout(g)
color_map = []

nxa.draw_networkx_edges(g,pos=position).interactive()+nxa.draw_networkx_nodes(g,pos=position,node_color='in_playlist',tooltip=['name']).properties(
    width=800,
    height=800,
    title='Instagram Reels Top Songs Artists'
).interactive()

alt.LayerChart(...)